In [1]:
from tqdm.auto import tqdm 
import pandas as pd
import numpy as np
import xarray as xr
import netCDF4 as nf
from netCDF4 import Dataset
%matplotlib inline
import glob
import cartopy.crs as ccrs
import matplotlib.pyplot as plt

In [2]:
datapath="/work/FAC/FGSE/IDYST/tbeucler/default/saranya/Data/ECMWF/ERA5_25kmx3hr/"
path="/work/FAC/FGSE/IDYST/tbeucler/default/saranya/create_ts/"
output="/work/FAC/FGSE/IDYST/tbeucler/default/saranya/create_ts/outputs/"
target="/work/FAC/FGSE/IDYST/tbeucler/default/saranya/create_ts/outputs/targets/"

In [3]:
tracklist = sorted(glob.glob(path+'tracks_wp/*2020*'))

In [4]:
maindf=pd.read_csv(output+'inner/inner_wp_all_vongfong.csv',delimiter=r",")
maindf.rename({"Unnamed: 0":"a"}, axis="columns", inplace=True)
maindf=maindf.drop('a', axis=1)

In [5]:
pdf_amphan=pd.read_csv(target+'2020_pmin_nwp_vongfong.csv',delimiter=r",")
pdf_amphan.rename({"Unnamed: 0":"a"}, axis="columns", inplace=True)
pdf_amphan=pdf_amphan.drop('a', axis=1)

In [6]:
maindf.shape

(81, 150)

In [7]:
extra=pd.read_csv(output+'nwp/outder_2020_nwp_vongfong.csv',delimiter=r",")

In [8]:
extra=pd.read_csv(output+'nwp/outder_2020_nwp_vongfong.csv',delimiter=r",")
extra.rename({"Unnamed: 0":"a"}, axis="columns", inplace=True)
extra=extra.drop('a', axis=1)

In [4]:
tracklist

['/work/FAC/FGSE/IDYST/tbeucler/default/saranya/create_ts/tracks_wp/nwp_20200508.csv',
 '/work/FAC/FGSE/IDYST/tbeucler/default/saranya/create_ts/tracks_wp/nwp_20201003.csv',
 '/work/FAC/FGSE/IDYST/tbeucler/default/saranya/create_ts/tracks_wp/nwp_20201016.csv',
 '/work/FAC/FGSE/IDYST/tbeucler/default/saranya/create_ts/tracks_wp/nwp_20201021.csv',
 '/work/FAC/FGSE/IDYST/tbeucler/default/saranya/create_ts/tracks_wp/nwp_20201025.csv',
 '/work/FAC/FGSE/IDYST/tbeucler/default/saranya/create_ts/tracks_wp/nwp_20201029.csv',
 '/work/FAC/FGSE/IDYST/tbeucler/default/saranya/create_ts/tracks_wp/nwp_20201108.csv']

In [5]:
dm1=xr.open_mfdataset([datapath+'/vor/vor_2020.nc',datapath+'/rhum/rhum_2020.nc'])

In [6]:
#ssingle level variables
def output_indices(TCtrack=None,ERA5date=None,ERA5hour=None):
    allindices = []
    for timeidx in range(len(TCtrack)):#len(track['time'])):
        datetrack,hourtrack = TCtrack['time'][timeidx].split(':')[0],TCtrack['time'][timeidx].split(':')[1][0:2]
        ####################################################################################################
        # Find the indices in ERA5 data with the same date as track
        ####################################################################################################
        dateind = []
        for ind,obj in enumerate(ERA5date):
            if obj==datetrack:
                dateind.append(ind)
        del ind,obj
        hourind = []
        hourextract = ERA5hour[int(np.min(np.asarray(dateind))):int(np.max(np.asarray(dateind)))+1]
        for ind,obj in enumerate(hourextract):
            if obj==hourtrack:            
                hourind.append(ind)
        allindices.append((int(np.min(np.asarray(dateind))),int(hourind[0])))
    return allindices

def extract_var(dataset=None,var='var138',indices=None):
    extractedvar = []
    for i in (range(len(indices))):
        realindex = indices[i][0]+indices[i][1]
        extractedvar.append(dataset[var][int(realindex),...].data)
    return np.asarray(extractedvar)

def largearea(dataset=None,invar=None,indices=None):
    if len(invar.shape) != 3:
        invar = np.squeeze(invar)
    ds = xr.Dataset(
    data_vars=dict(variable=(["time","lat","lon"], invar)),#mysvar[0])),
    coords=dict(lat=(["lat"], dataset.lat.data),lon=(["lon"], dataset.lon.data),time=(["time"], np.linspace(0,len(indices)-1,len(indices)))),
    attrs=dict(description="coords with matrices"),)
    
    var_out=np.zeros((len(indices),64,64))
    for it in range(len(indices)):
        latn, lats, lone, lonw = tc_orad[it,:]
        try:
            var_out[it,:,:]=ds['variable'][it,:,:].sel(lat=slice(lats,latn),lon=slice(lone,lonw))
        except:
            var_out[it,:,:]=ds['variable'][it,:,:].sel(lat=slice(lats,latn),lon=slice(lone,lonw))[0:64,0:64]
    return var_out

def largearea_withpres(dataset=None,invar=None,indices=None):
    ds = xr.Dataset(
    data_vars=dict(variable=(["time","plev","lat","lon"], invar)),#mysvar[0])),
    coords=dict(lat=(["lat"], dataset.lat.data),lon=(["lon"], dataset.lon.data),time=(["time"], np.linspace(0,len(indices)-1,len(indices))),
               plev=(["plev"],dataset.plev.data)),
    attrs=dict(description="coords with matrices"),)
    var_out=np.zeros((len(indices),len(dm1.plev.data),64,64))
    for it in range(len(indices)):
        latn, lats, lone, lonw = tc_orad[it,:]
        for ip in range(len(dm1.plev.data)):
            try:
                var_out[it,ip,:,:]=ds['variable'][it,ip,:,:].sel(lat=slice(lats,latn),lon=slice(lone,lonw))
            except:
                var_out[it,ip,:,:]=ds['variable'][it,ip,:,:].sel(lat=slice(lats,latn),lon=slice(lone,lonw))[0:64,0:64]
    return var_out 

In [7]:
###
def createmask(dm=None,irad=None,orad=None,lonselect=None,latselect=None):
    mask = []
    for ti in range(len(tc_irad)):
        lonselect = dm.lon.sel(lon=slice(orad[ti,:][2],orad[ti,:][3])).data
        latselect = np.flipud(dm.lat.sel(lat=slice(orad[ti,:][1],orad[ti,:][0])).data)
        if (lonselect.shape != 64) or (latselect.shape != 64):
            lon2d,lat2d = np.meshgrid(lonselect[0:64],latselect[0:64])
        else:
            lon2d,lat2d = np.meshgrid(lonselect,latselect)
        #############################################################################################
        latcriteria = np.logical_and(lat2d>irad[ti][0],lat2d<irad[ti][1])
        loncriteria = np.logical_and(lon2d>irad[ti][2],lon2d<irad[ti][3])
        allcriteria = np.logical_and(loncriteria,latcriteria)
        mask.append(allcriteria)
    return mask

def readyear(year=None):
    dm2 = xr.open_dataset(datapath+'/slev_vars/svars_'+str(year)+'.nc')
    tracklist = sorted(glob.glob('/work/FAC/FGSE/IDYST/tbeucler/default/saranya/create_ts/tracks_wp/*'+str(year)+'*'))
    era5_date = [str(dm2.time[i].data).split('T')[0] for i in range(len(dm2.time))]
    era5_hour = [str(dm2.time[i].data).split('T')[1][0:2] for i in range(len(dm2.time))]
    return dm2,tracklist,era5_date,era5_hour

dm2,tracklist,era5_date,era5_hour = readyear(2020)

TCs_ts = []
for TCobj in tqdm(tracklist):
    track=pd.read_csv(TCobj,delimiter=r",")
    lon1=track['lon'].to_numpy()
    lat1=track['lat'].to_numpy()
    pos = arr = np.stack((lat1, lon1), axis=1)
    ###########################################################################
    indices_store = output_indices(track,era5_date,era5_hour)
    ###########################################################################
    mysvar = [extract_var(dataset=dm2,var=obj,indices=indices_store) for obj in (list(dm2.keys()))]
    ###########################################################################
    tc_irad=np.empty((len(indices_store),4))
    tc_irad[:,0] = pos[:,0]-2
    tc_irad[:,1] = pos[:,0]+2
    tc_irad[:,2] = pos[:,1]-2
    tc_irad[:,3] = pos[:,1]+2
    
    tc_orad=np.empty((len(indices_store),4))
    tc_orad[:,0] = pos[:,0]-8
    tc_orad[:,1] = pos[:,0]+8
    tc_orad[:,2] = pos[:,1]-8
    tc_orad[:,3] = pos[:,1]+8
    ###########################################################################
    smallsvarout = [largearea(dm2,mysvar[i],indices_store) for i in (range(len(mysvar)))]
    svarname1 = ['outu10','outv10','out2mdewtmp','out2mtmp','outconv_ppt','outtot_cld_ice',\
           'outtot_cldwtr','outtot_cld_rain','outvi_div_cld_froz_wtr','outvi_div_cld_liq_wtr','outvi_div_gpot_flux',\
           'outvi_div_ke_flux','outvi_div_mass_flux','outvi_div_moisture_flux','outvi_div_olr_flux','outvi_div_tot_enrgy_flux',\
           'outvi_ke','outvi_pe_inte','outvi_pe_ie_latentenrgy','outvi_temp','outvi_olr','outvi_tot_enrgy','outvi_moisture_div']
   
    svarname = ['u10','v10','2mdewtmp','2mtmp','conv_ppt','tot_cld_ice',\
           'tot_cldwtr','tot_cld_rain','vi_div_cld_froz_wtr','vi_div_cld_liq_wtr','vi_div_gpot_flux',\
           'vi_div_ke_flux','vi_div_mass_flux','vi_div_moisture_flux','vi_div_olr_flux','vi_div_tot_enrgy_flux',\
           'vi_ke','vi_pe_inte','vi_pe_ie_latentenrgy','vi_temp','vi_olr','vi_tot_enrgy','vi_moisture_div']
    svardict = {varnameobj:varobj for (varnameobj,varobj) in zip(svarname,smallsvarout)}
    
    lonselect = dm2.lon.sel(lon=slice(tc_orad[0,:][2],tc_orad[0,:][3])).data
    latselect = np.flipud(dm2.lat.sel(lat=slice(tc_orad[0,:][1],tc_orad[0,:][0])).data)
    lon2d,lat2d = np.meshgrid(lonselect,latselect)

    mymask = createmask(dm=dm2,irad=tc_irad,orad=tc_orad,lonselect=lonselect,latselect=latselect)
    #############################################################################################
    tsdict = {}
    for ind,obj in (enumerate(svarname)):
        tslist = [svardict[svarname[ind]][i,...][~mymask[i]] for i in range(len(mymask))]
        tsdict[svarname1[ind]] = [np.nanmean(obj) for obj in tslist]
    #############################################################################################
    TCs_ts.append(tsdict)
#myvar = [extract_var(var=obj,indices=indices_store) for obj in vars_dm1]

  0%|          | 0/7 [00:00<?, ?it/s]

In [8]:
###radiation variables
def createmask(dm=None,irad=None,orad=None,lonselect=None,latselect=None):
    mask = []
    for ti in range(len(tc_irad)):
        lonselect = dm.lon.sel(lon=slice(orad[ti,:][2],orad[ti,:][3])).data
        latselect = np.flipud(dm.lat.sel(lat=slice(orad[ti,:][1],orad[ti,:][0])).data)
        if (lonselect.shape != 64) or (latselect.shape != 64):
            lon2d,lat2d = np.meshgrid(lonselect[0:64],latselect[0:64])
        else:
            lon2d,lat2d = np.meshgrid(lonselect,latselect)
        #############################################################################################
        latcriteria = np.logical_and(lat2d>irad[ti][0],lat2d<irad[ti][1])
        loncriteria = np.logical_and(lon2d>irad[ti][2],lon2d<irad[ti][3])
        allcriteria = np.logical_and(loncriteria,latcriteria)
        mask.append(allcriteria)
    return mask

def readyear(year=None):
    dm2 = xr.open_dataset(datapath+'/slev_vars/radvars_'+str(year)+'.nc')
    tracklist = sorted(glob.glob('/work/FAC/FGSE/IDYST/tbeucler/default/saranya/create_ts/tracks_wp/*'+str(year)+'*'))
    era5_date = [str(dm2.time[i].data).split('T')[0] for i in range(len(dm2.time))]
    era5_hour = [str(dm2.time[i].data).split('T')[1][0:2] for i in range(len(dm2.time))]
    return dm2,tracklist,era5_date,era5_hour

dm2,tracklist,era5_date,era5_hour = readyear(2020)

TCr_ts = []
for TCobj in tqdm(tracklist):
    track=pd.read_csv(TCobj,delimiter=r",")
    lon1=track['lon'].to_numpy()
    lat1=track['lat'].to_numpy()
    pos = arr = np.stack((lat1, lon1), axis=1)
    ###########################################################################
    indices_store = output_indices(track,era5_date,era5_hour)
    ###########################################################################
    mysvar = [extract_var(dataset=dm2,var=obj,indices=indices_store) for obj in (list(dm2.keys()))]
    ###########################################################################
    tc_irad=np.empty((len(indices_store),4))
    tc_irad[:,0] = pos[:,0]-2
    tc_irad[:,1] = pos[:,0]+2
    tc_irad[:,2] = pos[:,1]-2
    tc_irad[:,3] = pos[:,1]+2
    
    tc_orad=np.empty((len(indices_store),4))
    tc_orad[:,0] = pos[:,0]-8
    tc_orad[:,1] = pos[:,0]+8
    tc_orad[:,2] = pos[:,1]-8
    tc_orad[:,3] = pos[:,1]+8
    ###########################################################################
    smallsvarout = [largearea(dm2,mysvar[i],indices_store) for i in (range(len(mysvar)))]
    
    svarname1 = ['outcape','outinst_10m_wnd_gst','outinst_moisture_flux','outinst_ssh_flux','outsurfmean_swr_flux','outsurfmean_lhf',\
           'outsurfmean_lwr_flux','outsurfmean_shf','outdwnwrdmean_swr_flux','outtopmean_lwr_flux','outtopmean_swr_flux',\
           'outvimean_moisture_div','outsurf_lhf','outsurf_shf','outtot_suprcool_liqwtr','outtot_wtr_vpr']
    
    
    svarname = ['cape','inst_10m_wnd_gst','inst_moisture_flux','inst_ssh_flux','surfmean_swr_flux','surfmean_lhf',\
           'surfmean_lwr_flux','surfmean_shf','dwnwrdmean_swr_flux','topmean_lwr_flux','topmean_swr_flux',\
           'vimean_moisture_div','surf_lhf','surf_shf','tot_suprcool_liqwtr','tot_wtr_vpr']
    
    svardict = {varnameobj:varobj for (varnameobj,varobj) in zip(svarname,smallsvarout)}
    
    lonselect = dm2.lon.sel(lon=slice(tc_orad[0,:][2],tc_orad[0,:][3])).data
    latselect = np.flipud(dm2.lat.sel(lat=slice(tc_orad[0,:][1],tc_orad[0,:][0])).data)
    lon2d,lat2d = np.meshgrid(lonselect,latselect)

    mymask = createmask(dm=dm2,irad=tc_irad,orad=tc_orad,lonselect=lonselect,latselect=latselect)
    #############################################################################################
    tsdict = {}
    for ind,obj in (enumerate(svarname)):
        tslist = [svardict[svarname[ind]][i,...][~mymask[i]] for i in range(len(mymask))]
        tsdict[svarname1[ind]] = [np.nanmean(obj) for obj in tslist]
    #############################################################################################
    TCr_ts.append(tsdict)
#myvar = [extract_var(var=obj,indices=indices_store) for obj in vars_dm1]

  0%|          | 0/7 [00:00<?, ?it/s]

In [9]:
#########precip, mslp, sst variables############
def createmask(dm=None,irad=None,orad=None,lonselect=None,latselect=None):
    mask = []
    for ti in range(len(tc_irad)):
        lonselect = dm.lon.sel(lon=slice(orad[ti,:][2],orad[ti,:][3])).data
        latselect = np.flipud(dm.lat.sel(lat=slice(orad[ti,:][1],orad[ti,:][0])).data)
        if (lonselect.shape != 64) or (latselect.shape != 64):
            lon2d,lat2d = np.meshgrid(lonselect[0:64],latselect[0:64])
        else:
            lon2d,lat2d = np.meshgrid(lonselect,latselect)
        #############################################################################################
        latcriteria = np.logical_and(lat2d>irad[ti][0],lat2d<irad[ti][1])
        loncriteria = np.logical_and(lon2d>irad[ti][2],lon2d<irad[ti][3])
        allcriteria = np.logical_and(loncriteria,latcriteria)
        mask.append(allcriteria)
    return mask

def readyear(year=None):
    dm2 = xr.open_mfdataset([datapath+'/prate/prates_2020.nc',datapath+'/mslp/mslp_2020.nc',datapath+'/sst/sst_2020.nc'])
    tracklist = sorted(glob.glob('/work/FAC/FGSE/IDYST/tbeucler/default/saranya/create_ts/tracks_wp/*'+str(year)+'*'))
    era5_date = [str(dm2.time[i].data).split('T')[0] for i in range(len(dm2.time))]
    era5_hour = [str(dm2.time[i].data).split('T')[1][0:2] for i in range(len(dm2.time))]
    return dm2,tracklist,era5_date,era5_hour

dm2,tracklist,era5_date,era5_hour = readyear(2020)

TCpr_ts = []
for TCobj in tqdm(tracklist):
    track=pd.read_csv(TCobj,delimiter=r",")
    lon1=track['lon'].to_numpy()
    lat1=track['lat'].to_numpy()
    pos = arr = np.stack((lat1, lon1), axis=1)
    ###########################################################################
    indices_store = output_indices(track,era5_date,era5_hour)
    ###########################################################################
    mysvar = [extract_var(dataset=dm2,var=obj,indices=indices_store) for obj in (list(dm2.keys()))]
    ###########################################################################
    tc_irad=np.empty((len(indices_store),4))
    tc_irad[:,0] = pos[:,0]-2
    tc_irad[:,1] = pos[:,0]+2
    tc_irad[:,2] = pos[:,1]-2
    tc_irad[:,3] = pos[:,1]+2
    
    tc_orad=np.empty((len(indices_store),4))
    tc_orad[:,0] = pos[:,0]-8
    tc_orad[:,1] = pos[:,0]+8
    tc_orad[:,2] = pos[:,1]-8
    tc_orad[:,3] = pos[:,1]+8
    ###########################################################################
    smallsvarout = [largearea(dm2,mysvar[i],indices_store) for i in (range(len(mysvar)))]
    
    svarname1 = ['outmslp','outconv_rrate','outls_rrate','outmn_conv_prate',\
                 'outmn_ls_prate','outmn_tot_prate','outsst']
    
    svarname = ['mslp','conv_rrate','ls_rrate','mn_conv_prate','mn_ls_prate','mn_tot_prate','sst']
    
    svardict = {varnameobj:varobj for (varnameobj,varobj) in zip(svarname,smallsvarout)}
    
    lonselect = dm2.lon.sel(lon=slice(tc_orad[0,:][2],tc_orad[0,:][3])).data
    latselect = np.flipud(dm2.lat.sel(lat=slice(tc_orad[0,:][1],tc_orad[0,:][0])).data)
    lon2d,lat2d = np.meshgrid(lonselect,latselect)

    mymask = createmask(dm=dm2,irad=tc_irad,orad=tc_orad,lonselect=lonselect,latselect=latselect)
    #############################################################################################
    tsdict = {}
    for ind,obj in (enumerate(svarname)):
        tslist = [svardict[svarname[ind]][i,...][~mymask[i]] for i in range(len(mymask))]
        tsdict[svarname1[ind]] = [np.nanmean(obj) for obj in tslist]
    #############################################################################################
    TCpr_ts.append(tsdict)
#myvar = [extract_var(var=obj,indices=indices_store) for obj in vars_dm1]

  0%|          | 0/7 [00:00<?, ?it/s]

In [10]:
rlev_vongfong=pd.DataFrame.from_dict(TCr_ts[0])

In [11]:
prlev_vongfong=pd.DataFrame.from_dict(TCpr_ts[0])

In [12]:
slev_vongfong=pd.DataFrame.from_dict(TCs_ts[0])

In [18]:
prlev_vongfong['outmslp']=prlev_vongfong['outmslp']/100

In [19]:
ts1_vongfong=pd.concat([maindf,extra,slev_vongfong,rlev_vongfong,prlev_vongfong], axis=1, join='inner')

In [20]:
ts1_vongfong

,w10max,div100,div200,div250,div300,div400,div500,eqt1000,eqt200,eqt250,...,outsurf_shf,outtot_suprcool_liqwtr,outtot_wtr_vpr,outmslp,outconv_rrate,outls_rrate,outmn_conv_prate,outmn_ls_prate,outmn_tot_prate,outsst
0,10.309203,-0.000001,-0.000005,5.563559e-06,7.814137e-06,5.854761e-06,-6.114945e-06,336.610168,401.203491,401.203491,...,-36384.133333,0.024252,55.648470,1007.912400,0.000027,0.000017,0.000024,0.000018,0.000042,302.781632
1,10.910505,0.000008,-0.000003,-3.635838e-06,-9.401367e-07,7.783531e-07,4.811327e-06,337.293274,400.864471,400.864471,...,-23060.050000,0.027624,55.776591,1008.591534,0.000036,0.000017,0.000031,0.000016,0.000047,302.792321
2,9.938855,0.000017,0.000018,1.688236e-06,-3.315896e-06,-8.495769e-06,2.562160e-06,336.432892,399.526886,399.526886,...,-23291.116667,0.024721,54.846309,1010.167878,0.000038,0.000023,0.000035,0.000022,0.000057,302.794176
3,8.958860,0.000019,0.000022,6.507719e-07,-4.473000e-06,5.714251e-07,6.604291e-06,335.481506,399.772156,399.772156,...,-27428.983333,0.028959,55.152194,1010.449195,0.000056,0.000033,0.000043,0.000031,0.000074,302.805685
4,9.296708,0.000013,0.000016,-7.199931e-06,-6.999429e-06,6.906871e-06,7.085467e-06,336.512817,399.554321,399.554321,...,-37609.550000,0.027555,54.970604,1008.707179,0.000090,0.000037,0.000088,0.000037,0.000124,302.808175
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76,12.185334,0.000024,0.000012,8.853050e-06,3.857888e-06,-1.038136e-05,-1.042617e-05,328.694977,402.119659,402.119659,...,-5708.020747,0.027698,48.393827,1005.372844,0.000031,0.000010,0.000030,0.000009,0.000039,300.874972
77,12.407811,0.000016,-0.000004,3.084240e-06,3.603419e-06,-5.541900e-06,1.467028e-06,326.974548,402.748444,402.748444,...,-6569.566667,0.024993,48.717565,1005.447826,0.000050,0.000016,0.000049,0.000015,0.000064,300.820306
78,12.184855,0.000025,0.000002,-2.187550e-06,2.550591e-06,-5.712461e-06,1.519241e-06,325.978271,402.592041,402.592041,...,-14938.600000,0.030974,48.915104,1006.722231,0.000078,0.000032,0.000069,0.000028,0.000096,300.494206
79,13.254792,0.000017,0.000002,-9.022481e-07,1.410654e-06,-4.320523e-06,-2.568056e-07,322.068451,403.182495,403.182495,...,-41499.900000,0.028081,48.799441,1006.363004,0.000071,0.000047,0.000081,0.000045,0.000127,300.021130


In [13]:
#Pressure vars - vor hum
TCp_ts = []
for TCobj in tqdm(tracklist):
    track=pd.read_csv(TCobj,delimiter=r",")
    lon1=track['lon'].to_numpy()
    lat1=track['lat'].to_numpy()
    pos = arr = np.stack((lat1, lon1), axis=1)
    ###########################################################################
    indices_store = output_indices(track,era5_date,era5_hour)
    ###########################################################################
    mypvar = [extract_var(dataset=dm1,var=obj,indices=indices_store) for obj in (list(dm1.keys()))]
    ###########################################################################
    tc_irad=np.empty((len(indices_store),4))
    tc_irad[:,0] = pos[:,0]-2
    tc_irad[:,1] = pos[:,0]+2
    tc_irad[:,2] = pos[:,1]-2
    tc_irad[:,3] = pos[:,1]+2
    
    tc_orad=np.empty((len(indices_store),4))
    tc_orad[:,0] = pos[:,0]-8
    tc_orad[:,1] = pos[:,0]+8
    tc_orad[:,2] = pos[:,1]-8
    tc_orad[:,3] = pos[:,1]+8
    ###########################################################################
    smallpvarout = [largearea_withpres(dm1,mypvar[i],indices_store) for i in (range(len(mypvar)))]    
    pvarname1 = ['outvort','outrhum']
    pvarname = ['vort','rhum']
    pvardict = {varnameobj:varobj for (varnameobj,varobj) in zip(pvarname,smallpvarout)}
    
    lonselect = dm1.lon.sel(lon=slice(tc_orad[0,:][2],tc_orad[0,:][3])).data
    latselect = np.flipud(dm1.lat.sel(lat=slice(tc_orad[0,:][1],tc_orad[0,:][0])).data)
    lon2d,lat2d = np.meshgrid(lonselect,latselect)
    mymask = createmask(dm=dm1,irad=tc_irad,orad=tc_orad,lonselect=lonselect,latselect=latselect)
    #############################################################################################
    ts_pdict = {}
    for ind,obj in (enumerate(pvarname)):
        pvarTS_store = []
        for plevv in range(len(dm1.plev.data)):
            tempvar = pvardict[pvarname[ind]][:,plevv,...]
            tempts = [tempvar[i,...][~mymask[i]] for i in range(len(mymask))]
            tempTSERIES = [np.nanmean(obj) for obj in tempts]
            pvarTS_store.append(tempTSERIES)
        ts_pdict[pvarname1[ind]] = np.asarray(pvarTS_store).transpose()
    #############################################################################################
    TCp_ts.append(ts_pdict)
#myvar = [extract_var(var=obj,indices=indices_store) for obj in vars_dm1]

  0%|          | 0/7 [00:00<?, ?it/s]

In [14]:
dm1=xr.open_dataset(datapath+'/gpot/gpot_2020.nc')

In [15]:
# Pressure vars - geopotential
TCgp_ts = []
for TCobj in tqdm(tracklist):
    track=pd.read_csv(TCobj,delimiter=r",")
    lon1=track['lon'].to_numpy()
    lat1=track['lat'].to_numpy()
    pos = arr = np.stack((lat1, lon1), axis=1)
    ###########################################################################
    indices_store = output_indices(track,era5_date,era5_hour)
    ###########################################################################
    mypvar = [extract_var(dataset=dm1,var=obj,indices=indices_store) for obj in (list(dm1.keys()))]
    ###########################################################################
    tc_irad=np.empty((len(indices_store),4))
    tc_irad[:,0] = pos[:,0]-2
    tc_irad[:,1] = pos[:,0]+2
    tc_irad[:,2] = pos[:,1]-2
    tc_irad[:,3] = pos[:,1]+2
    
    tc_orad=np.empty((len(indices_store),4))
    tc_orad[:,0] = pos[:,0]-8
    tc_orad[:,1] = pos[:,0]+8
    tc_orad[:,2] = pos[:,1]-8
    tc_orad[:,3] = pos[:,1]+8
    ###########################################################################
    smallpvarout = [largearea_withpres(dm1,mypvar[i],indices_store) for i in (range(len(mypvar)))]    
    pvarname1 = ['outgpot']
    pvarname = ['gpot']
    pvardict = {varnameobj:varobj for (varnameobj,varobj) in zip(pvarname,smallpvarout)}
    
    lonselect = dm1.lon.sel(lon=slice(tc_orad[0,:][2],tc_orad[0,:][3])).data
    latselect = np.flipud(dm1.lat.sel(lat=slice(tc_orad[0,:][1],tc_orad[0,:][0])).data)
    lon2d,lat2d = np.meshgrid(lonselect,latselect)
    mymask = createmask(dm=dm1,irad=tc_irad,orad=tc_orad,lonselect=lonselect,latselect=latselect)
    #############################################################################################
    ts_pdict = {}
    for ind,obj in (enumerate(pvarname)):
        pvarTS_store = []
        for plevv in range(len(dm1.plev.data)):
            tempvar = pvardict[pvarname[ind]][:,plevv,...]
            tempts = [tempvar[i,...][~mymask[i]] for i in range(len(mymask))]
            tempTSERIES = [np.nanmean(obj) for obj in tempts]
            pvarTS_store.append(tempTSERIES)
        ts_pdict[pvarname1[ind]] = np.asarray(pvarTS_store).transpose()
    #############################################################################################
    TCgp_ts.append(ts_pdict)
#myvar = [extract_var(var=obj,indices=indices_store) for obj in vars_dm1]

  0%|          | 0/7 [00:00<?, ?it/s]

In [16]:
dm1=xr.open_dataset(datapath+'/vvel/w_wnd_2020.nc')

In [17]:
#Pressure vars - vertical velocity
TCvv_ts = []
for TCobj in tqdm(tracklist):
    track=pd.read_csv(TCobj,delimiter=r",")
    lon1=track['lon'].to_numpy()
    lat1=track['lat'].to_numpy()
    pos = arr = np.stack((lat1, lon1), axis=1)
    ###########################################################################
    indices_store = output_indices(track,era5_date,era5_hour)
    ###########################################################################
    mypvar = [extract_var(dataset=dm1,var=obj,indices=indices_store) for obj in (list(dm1.keys()))]
    ###########################################################################
    tc_irad=np.empty((len(indices_store),4))
    tc_irad[:,0] = pos[:,0]-2
    tc_irad[:,1] = pos[:,0]+2
    tc_irad[:,2] = pos[:,1]-2
    tc_irad[:,3] = pos[:,1]+2
    
    tc_orad=np.empty((len(indices_store),4))
    tc_orad[:,0] = pos[:,0]-8
    tc_orad[:,1] = pos[:,0]+8
    tc_orad[:,2] = pos[:,1]-8
    tc_orad[:,3] = pos[:,1]+8
    ###########################################################################
    smallpvarout = [largearea_withpres(dm1,mypvar[i],indices_store) for i in (range(len(mypvar)))]    
    pvarname1 = ['outvvel']
    pvarname = ['vvel']
    pvardict = {varnameobj:varobj for (varnameobj,varobj) in zip(pvarname,smallpvarout)}
    
    lonselect = dm1.lon.sel(lon=slice(tc_orad[0,:][2],tc_orad[0,:][3])).data
    latselect = np.flipud(dm1.lat.sel(lat=slice(tc_orad[0,:][1],tc_orad[0,:][0])).data)
    lon2d,lat2d = np.meshgrid(lonselect,latselect)
    mymask = createmask(dm=dm1,irad=tc_irad,orad=tc_orad,lonselect=lonselect,latselect=latselect)
    #############################################################################################
    ts_pdict = {}
    for ind,obj in (enumerate(pvarname)):
        pvarTS_store = []
        for plevv in range(len(dm1.plev.data)):
            tempvar = pvardict[pvarname[ind]][:,plevv,...]
            tempts = [tempvar[i,...][~mymask[i]] for i in range(len(mymask))]
            tempTSERIES = [np.nanmean(obj) for obj in tempts]
            pvarTS_store.append(tempTSERIES)
        ts_pdict[pvarname1[ind]] = np.asarray(pvarTS_store).transpose()
    #############################################################################################
    TCvv_ts.append(ts_pdict)
#myvar = [extract_var(var=obj,indices=indices_store) for obj in vars_dm1]

  0%|          | 0/7 [00:00<?, ?it/s]

In [18]:
preslv = [str(int(obj)) for obj in dm1.plev.data/100]

In [19]:
dx=xr.open_dataset(datapath+'/tlevs/tlevs_2020.nc')

In [20]:
plev=[str(int(obj)) for obj in dx.plev.data/100]

In [22]:
#storm1
#storm 1 vongfong
maindf=pd.read_csv(output+'inner/inner_wp_all_vongfong.csv',delimiter=r",")
maindf.rename({"Unnamed: 0":"a"}, axis="columns", inplace=True)
maindf=maindf.drop('a', axis=1)

pdf_vongfong=pd.read_csv(target+'2020_pmin_nwp_vongfong.csv',delimiter=r",")
pdf_vongfong.rename({"Unnamed: 0":"a"}, axis="columns", inplace=True)
pdf_vongfong=pdf_vongfong.drop('a', axis=1)

extra=pd.read_csv(output+'nwp/outder_2020_nwp_vongfong.csv',delimiter=r",")
extra.rename({"Unnamed: 0":"a"}, axis="columns", inplace=True)
extra=extra.drop('a', axis=1)

rlev_vongfong=pd.DataFrame.from_dict(TCr_ts[0])
prlev_vongfong=pd.DataFrame.from_dict(TCpr_ts[0])
slev_vongfong=pd.DataFrame.from_dict(TCs_ts[0])

prlev_vongfong['outmslp']=prlev_vongfong['outmslp']/100

ts1_vongfong=pd.concat([maindf,extra,slev_vongfong,rlev_vongfong,prlev_vongfong], axis=1, join='inner')

tempvortdict = {'outvort_'+plev[i]:TCp_ts[0]['outvort'][:,i] for i in range(21)}
temprhumdict = {'outrhum_'+plev[i]:TCp_ts[0]['outrhum'][:,i] for i in range(21)}
tempgpotdict = {'outgpot_'+plev[i]:TCgp_ts[0]['outgpot'][:,i] for i in range(21)}
tempvveldict = {'outvvel_'+preslv[i]:TCvv_ts[0]['outvvel'][:,i] for i in range(18)}
dict1 = {**tempvortdict,**temprhumdict}
dict3 = {**tempgpotdict,**tempvveldict}

alldict = {**dict1,**dict3}
plev_vongfong=pd.DataFrame.from_dict(alldict)
ts_vongfong=pd.concat([pdf_vongfong,ts1_vongfong,plev_vongfong], axis=1, join='inner')
ts_vongfong

,Pmin,w10max,div100,div200,div250,div300,div400,div500,eqt1000,eqt200,...,outvvel_250,outvvel_300,outvvel_400,outvvel_500,outvvel_600,outvvel_700,outvvel_850,outvvel_925,outvvel_975,outvvel_1000
0,1006.503125,10.309203,-0.000001,-0.000005,5.563559e-06,7.814137e-06,5.854761e-06,-6.114945e-06,336.610168,401.203491,...,-0.017239,-0.014685,-0.014093,-0.021459,-0.023367,-0.023218,-0.016347,-0.012411,-0.011369,-0.011921
1,1007.219375,10.910505,0.000008,-0.000003,-3.635838e-06,-9.401367e-07,7.783531e-07,4.811327e-06,337.293274,400.864471,...,-0.007932,-0.007196,-0.010029,-0.005048,0.005829,0.004892,0.009933,0.011347,0.012184,0.012573
2,1008.651250,9.938855,0.000017,0.000018,1.688236e-06,-3.315896e-06,-8.495769e-06,2.562160e-06,336.432892,399.526886,...,-0.008368,-0.011420,-0.010271,-0.021445,-0.026553,-0.031266,-0.014306,0.001209,0.013136,0.019182
3,1009.249375,8.958860,0.000019,0.000022,6.507719e-07,-4.473000e-06,5.714251e-07,6.604291e-06,335.481506,399.772156,...,-0.014817,-0.021382,-0.014912,-0.021877,-0.032610,-0.043306,-0.041707,-0.033292,-0.020583,-0.013190
4,1007.211875,9.296708,0.000013,0.000016,-7.199931e-06,-6.999429e-06,6.906871e-06,7.085467e-06,336.512817,399.554321,...,-0.016345,-0.026049,-0.023159,-0.024482,-0.030461,-0.041564,-0.040362,-0.032811,-0.018987,-0.011025
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76,1001.025625,12.185334,0.000024,0.000012,8.853050e-06,3.857888e-06,-1.038136e-05,-1.042617e-05,328.694977,402.119659,...,-0.061413,-0.062718,-0.049377,-0.023853,-0.011261,-0.008431,-0.008181,-0.000613,-0.000111,-0.000875
77,1000.696875,12.407811,0.000016,-0.000004,3.084240e-06,3.603419e-06,-5.541900e-06,1.467028e-06,326.974548,402.748444,...,-0.088857,-0.099130,-0.101710,-0.086532,-0.068555,-0.047266,-0.013787,0.009541,0.018420,0.021655
78,1002.281250,12.184855,0.000025,0.000002,-2.187550e-06,2.550591e-06,-5.712461e-06,1.519241e-06,325.978271,402.592041,...,-0.077323,-0.084952,-0.096958,-0.095587,-0.086664,-0.077748,-0.038221,-0.005653,0.004849,0.006095
79,1001.694375,13.254792,0.000017,0.000002,-9.022481e-07,1.410654e-06,-4.320523e-06,-2.568056e-07,322.068451,403.182495,...,-0.081520,-0.084607,-0.092742,-0.092829,-0.090708,-0.088155,-0.056129,-0.029039,-0.021095,-0.018348


In [23]:
df1=ts_vongfong
df1=df1.drop('ws10', axis=1)
df1=df1.drop('u10', axis=1)
df1=df1.drop('v10', axis=1)
df1=df1.drop('out_ws10', axis=1)
df1=df1.drop('outu10', axis=1)
df1=df1.drop('outv10', axis=1)
df1=df1.drop('vi_div_ke_flux', axis=1)
df1=df1.drop('vi_div_gpot_flux', axis=1)
df1=df1.drop('vi_ke', axis=1)
df1=df1.drop('vi_div_olr_flux', axis=1)
df1=df1.drop('vi_temp', axis=1)
df1=df1.drop('inst_10m_wnd_gst', axis=1)
df1=df1.drop('surfmean_swr_flux', axis=1)
df1=df1.drop('surfmean_lwr_flux', axis=1)
df1=df1.drop('dwnwrdmean_swr_flux', axis=1)
df1=df1.drop('topmean_lwr_flux', axis=1)
df1=df1.drop('topmean_swr_flux', axis=1)
df1=df1.drop('outvi_div_ke_flux', axis=1)
df1=df1.drop('outvi_div_gpot_flux', axis=1)
df1=df1.drop('outvi_ke', axis=1)
df1=df1.drop('outvi_div_olr_flux', axis=1)
df1=df1.drop('outvi_temp', axis=1)
df1=df1.drop('outinst_10m_wnd_gst', axis=1)
df1=df1.drop('outsurfmean_swr_flux', axis=1)
df1=df1.drop('outsurfmean_lwr_flux', axis=1)
df1=df1.drop('outdwnwrdmean_swr_flux', axis=1)
df1=df1.drop('outtopmean_lwr_flux', axis=1)
df1=df1.drop('outtopmean_swr_flux', axis=1)

In [24]:
df1=df1.drop('outmslp', axis=1)
df1=df1.drop('mslp', axis=1)

In [25]:
df1

,Pmin,w10max,div100,div200,div250,div300,div400,div500,eqt1000,eqt200,...,outvvel_250,outvvel_300,outvvel_400,outvvel_500,outvvel_600,outvvel_700,outvvel_850,outvvel_925,outvvel_975,outvvel_1000
0,1006.503125,10.309203,-0.000001,-0.000005,5.563559e-06,7.814137e-06,5.854761e-06,-6.114945e-06,336.610168,401.203491,...,-0.017239,-0.014685,-0.014093,-0.021459,-0.023367,-0.023218,-0.016347,-0.012411,-0.011369,-0.011921
1,1007.219375,10.910505,0.000008,-0.000003,-3.635838e-06,-9.401367e-07,7.783531e-07,4.811327e-06,337.293274,400.864471,...,-0.007932,-0.007196,-0.010029,-0.005048,0.005829,0.004892,0.009933,0.011347,0.012184,0.012573
2,1008.651250,9.938855,0.000017,0.000018,1.688236e-06,-3.315896e-06,-8.495769e-06,2.562160e-06,336.432892,399.526886,...,-0.008368,-0.011420,-0.010271,-0.021445,-0.026553,-0.031266,-0.014306,0.001209,0.013136,0.019182
3,1009.249375,8.958860,0.000019,0.000022,6.507719e-07,-4.473000e-06,5.714251e-07,6.604291e-06,335.481506,399.772156,...,-0.014817,-0.021382,-0.014912,-0.021877,-0.032610,-0.043306,-0.041707,-0.033292,-0.020583,-0.013190
4,1007.211875,9.296708,0.000013,0.000016,-7.199931e-06,-6.999429e-06,6.906871e-06,7.085467e-06,336.512817,399.554321,...,-0.016345,-0.026049,-0.023159,-0.024482,-0.030461,-0.041564,-0.040362,-0.032811,-0.018987,-0.011025
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76,1001.025625,12.185334,0.000024,0.000012,8.853050e-06,3.857888e-06,-1.038136e-05,-1.042617e-05,328.694977,402.119659,...,-0.061413,-0.062718,-0.049377,-0.023853,-0.011261,-0.008431,-0.008181,-0.000613,-0.000111,-0.000875
77,1000.696875,12.407811,0.000016,-0.000004,3.084240e-06,3.603419e-06,-5.541900e-06,1.467028e-06,326.974548,402.748444,...,-0.088857,-0.099130,-0.101710,-0.086532,-0.068555,-0.047266,-0.013787,0.009541,0.018420,0.021655
78,1002.281250,12.184855,0.000025,0.000002,-2.187550e-06,2.550591e-06,-5.712461e-06,1.519241e-06,325.978271,402.592041,...,-0.077323,-0.084952,-0.096958,-0.095587,-0.086664,-0.077748,-0.038221,-0.005653,0.004849,0.006095
79,1001.694375,13.254792,0.000017,0.000002,-9.022481e-07,1.410654e-06,-4.320523e-06,-2.568056e-07,322.068451,403.182495,...,-0.081520,-0.084607,-0.092742,-0.092829,-0.090708,-0.088155,-0.056129,-0.029039,-0.021095,-0.018348


In [46]:
var_names=df1.columns.values.tolist()
var_names

['Pmin',
 'w10max',
 'div100',
 'div200',
 'div250',
 'div300',
 'div400',
 'div500',
 'eqt1000',
 'eqt200',
 'eqt250',
 'eqt300',
 'eqt400',
 'eqt500',
 'eqt600',
 'eqt700',
 'eqt850',
 'eqt925',
 'sh850_200',
 'sh850_250',
 'sh925_200',
 'sh925_250',
 'wstrss10',
 '2mdewtmp',
 '2mtmp',
 'conv_ppt',
 'tot_cld_ice',
 'tot_cldwtr',
 'tot_cld_rain',
 'vi_div_cld_froz_wtr',
 'vi_div_cld_liq_wtr',
 'vi_div_mass_flux',
 'vi_div_moisture_flux',
 'vi_div_tot_enrgy_flux',
 'vi_pe_inte',
 'vi_pe_ie_latentenrgy',
 'vi_olr',
 'vi_tot_enrgy',
 'vi_moisture_div',
 'cape',
 'inst_moisture_flux',
 'inst_ssh_flux',
 'surfmean_lhf',
 'surfmean_shf',
 'vimean_moisture_div',
 'surf_lhf',
 'surf_shf',
 'tot_suprcool_liqwtr',
 'tot_wtr_vpr',
 'conv_rrate',
 'ls_rrate',
 'mn_conv_prate',
 'mn_ls_prate',
 'mn_tot_prate',
 'sst',
 'vort_10',
 'vort_20',
 'vort_30',
 'vort_50',
 'vort_70',
 'vort_100',
 'vort_150',
 'vort_200',
 'vort_250',
 'vort_300',
 'vort_400',
 'vort_500',
 'vort_600',
 'vort_700',
 'vor

In [26]:
df1.to_csv(output+'timeseries_all_wp_vongfong.csv')

In [ ]:
#removed tlevs, pvor, mslps, wind vars except vorticity

In [27]:
#Storms 2-
maindf2=pd.read_csv(output+'inner/inner_wp_all_chanhom.csv',delimiter=r",")
maindf2.rename({"Unnamed: 0":"a"}, axis="columns", inplace=True)
maindf2=maindf2.drop('a', axis=1)

pdf_chanhom=pd.read_csv(target+'2020_pmin_nwp_chanhom.csv',delimiter=r",")
pdf_chanhom.rename({"Unnamed: 0":"a"}, axis="columns", inplace=True)
pdf_chanhom=pdf_chanhom.drop('a', axis=1)

extra=pd.read_csv(output+'nwp/outder_2020_nwp_chanhom.csv',delimiter=r",")
extra.rename({"Unnamed: 0":"a"}, axis="columns", inplace=True)
extra=extra.drop('a', axis=1)

In [28]:
rlev_chanhom=pd.DataFrame.from_dict(TCr_ts[1])
prlev_chanhom=pd.DataFrame.from_dict(TCpr_ts[1])
slev_chanhom=pd.DataFrame.from_dict(TCs_ts[1])

prlev_chanhom['outmslp']=prlev_chanhom['outmslp']/100

ts1_chanhom=pd.concat([maindf2,extra,slev_chanhom,rlev_chanhom,prlev_chanhom], axis=1, join='inner')

In [29]:
#storm2
tempvortdict = {'outvort_'+plev[i]:TCp_ts[1]['outvort'][:,i] for i in range(21)}
temprhumdict = {'outrhum_'+plev[i]:TCp_ts[1]['outrhum'][:,i] for i in range(21)}
tempgpotdict = {'outgpot_'+plev[i]:TCgp_ts[1]['outgpot'][:,i] for i in range(21)}
tempvveldict = {'outvvel_'+preslv[i]:TCvv_ts[1]['outvvel'][:,i] for i in range(18)}
dict1 = {**tempvortdict,**temprhumdict}
dict3 = {**tempgpotdict,**tempvveldict}

alldict = {**dict1,**dict3}

In [30]:
plev_chanhom=pd.DataFrame.from_dict(alldict)
ts_chanhom=pd.concat([pdf_chanhom,ts1_chanhom,plev_chanhom], axis=1, join='inner')
ts_chanhom

,Pmin,w10max,div100,div200,div250,div300,div400,div500,eqt1000,eqt200,...,outvvel_250,outvvel_300,outvvel_400,outvvel_500,outvvel_600,outvvel_700,outvvel_850,outvvel_925,outvvel_975,outvvel_1000
0,1002.993750,9.376366,2.092879e-05,0.000003,-9.492625e-07,1.596399e-07,-1.751216e-06,-3.277715e-06,338.670441,354.683838,...,-0.062386,-0.060082,-0.063255,-0.053828,-0.042080,-0.034860,-0.025159,-0.023146,-0.014330,-0.008741
1,1003.635625,9.295533,1.044931e-05,0.000010,9.458436e-06,-4.367001e-06,-3.945375e-06,1.877123e-06,339.252808,354.368103,...,-0.052425,-0.042520,-0.040097,-0.020129,-0.002288,0.005094,0.004730,0.002989,0.009007,0.013176
2,1005.323750,10.131128,1.641898e-05,0.000015,7.985401e-06,-5.679445e-06,-5.566510e-06,2.914433e-06,338.594604,354.061829,...,-0.049548,-0.036038,-0.025891,-0.012063,-0.003112,-0.003028,-0.016290,-0.015776,-0.005011,0.001739
3,1004.497500,10.922950,1.843215e-05,0.000016,3.257839e-06,-2.021105e-06,7.119737e-07,4.734732e-06,338.228210,353.762207,...,-0.042003,-0.027758,-0.014547,-0.021700,-0.025220,-0.033379,-0.048930,-0.041358,-0.027003,-0.018426
4,1002.913750,10.921099,1.746406e-05,0.000015,-3.864217e-06,-1.012786e-05,2.474067e-06,2.514803e-06,338.604980,353.883606,...,-0.040597,-0.031724,-0.019392,-0.036644,-0.048352,-0.059647,-0.055391,-0.036099,-0.016038,-0.004918
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
105,1010.381250,15.260516,6.882865e-06,0.000001,6.311119e-06,4.303548e-06,-8.145852e-06,-1.798526e-08,307.860107,349.829712,...,0.001963,0.008806,-0.032558,-0.053620,-0.060068,-0.034669,-0.014259,-0.003236,0.000338,0.001661
106,1011.150000,14.735302,1.320104e-07,-0.000006,-2.592002e-06,1.388817e-06,4.577279e-06,3.143935e-06,308.141876,349.570618,...,-0.005751,-0.015676,-0.066960,-0.084195,-0.076993,-0.058855,-0.012383,0.002143,0.003600,0.003575
107,1010.911250,14.472641,8.215622e-06,-0.000002,-3.117607e-07,6.945216e-06,-3.222564e-07,-7.365141e-07,310.461548,349.037231,...,-0.011831,-0.029064,-0.079038,-0.099321,-0.084296,-0.056213,-0.022702,-0.011650,-0.005753,-0.003370
108,1010.197500,13.135821,-8.778028e-07,0.000008,-3.190881e-06,2.965728e-08,-3.890142e-06,-2.785965e-06,308.148254,348.835663,...,-0.010985,-0.025936,-0.060773,-0.088269,-0.065035,-0.024801,0.001298,0.004623,0.005371,0.005806


In [31]:
df1=ts_chanhom
df1=df1.drop('ws10', axis=1)
df1=df1.drop('u10', axis=1)
df1=df1.drop('v10', axis=1)
df1=df1.drop('out_ws10', axis=1)
df1=df1.drop('outu10', axis=1)
df1=df1.drop('outv10', axis=1)
df1=df1.drop('vi_div_ke_flux', axis=1)
df1=df1.drop('vi_div_gpot_flux', axis=1)
df1=df1.drop('vi_ke', axis=1)
df1=df1.drop('vi_div_olr_flux', axis=1)
df1=df1.drop('vi_temp', axis=1)
df1=df1.drop('inst_10m_wnd_gst', axis=1)
df1=df1.drop('surfmean_swr_flux', axis=1)
df1=df1.drop('surfmean_lwr_flux', axis=1)
df1=df1.drop('dwnwrdmean_swr_flux', axis=1)
df1=df1.drop('topmean_lwr_flux', axis=1)
df1=df1.drop('topmean_swr_flux', axis=1)
df1=df1.drop('outvi_div_ke_flux', axis=1)
df1=df1.drop('outvi_div_gpot_flux', axis=1)
df1=df1.drop('outvi_ke', axis=1)
df1=df1.drop('outvi_div_olr_flux', axis=1)
df1=df1.drop('outvi_temp', axis=1)
df1=df1.drop('outinst_10m_wnd_gst', axis=1)
df1=df1.drop('outsurfmean_swr_flux', axis=1)
df1=df1.drop('outsurfmean_lwr_flux', axis=1)
df1=df1.drop('outdwnwrdmean_swr_flux', axis=1)
df1=df1.drop('outtopmean_lwr_flux', axis=1)
df1=df1.drop('outtopmean_swr_flux', axis=1)
df1=df1.drop('outmslp', axis=1)
df1=df1.drop('mslp', axis=1)

In [32]:
df1

,Pmin,w10max,div100,div200,div250,div300,div400,div500,eqt1000,eqt200,...,outvvel_250,outvvel_300,outvvel_400,outvvel_500,outvvel_600,outvvel_700,outvvel_850,outvvel_925,outvvel_975,outvvel_1000
0,1002.993750,9.376366,2.092879e-05,0.000003,-9.492625e-07,1.596399e-07,-1.751216e-06,-3.277715e-06,338.670441,354.683838,...,-0.062386,-0.060082,-0.063255,-0.053828,-0.042080,-0.034860,-0.025159,-0.023146,-0.014330,-0.008741
1,1003.635625,9.295533,1.044931e-05,0.000010,9.458436e-06,-4.367001e-06,-3.945375e-06,1.877123e-06,339.252808,354.368103,...,-0.052425,-0.042520,-0.040097,-0.020129,-0.002288,0.005094,0.004730,0.002989,0.009007,0.013176
2,1005.323750,10.131128,1.641898e-05,0.000015,7.985401e-06,-5.679445e-06,-5.566510e-06,2.914433e-06,338.594604,354.061829,...,-0.049548,-0.036038,-0.025891,-0.012063,-0.003112,-0.003028,-0.016290,-0.015776,-0.005011,0.001739
3,1004.497500,10.922950,1.843215e-05,0.000016,3.257839e-06,-2.021105e-06,7.119737e-07,4.734732e-06,338.228210,353.762207,...,-0.042003,-0.027758,-0.014547,-0.021700,-0.025220,-0.033379,-0.048930,-0.041358,-0.027003,-0.018426
4,1002.913750,10.921099,1.746406e-05,0.000015,-3.864217e-06,-1.012786e-05,2.474067e-06,2.514803e-06,338.604980,353.883606,...,-0.040597,-0.031724,-0.019392,-0.036644,-0.048352,-0.059647,-0.055391,-0.036099,-0.016038,-0.004918
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
105,1010.381250,15.260516,6.882865e-06,0.000001,6.311119e-06,4.303548e-06,-8.145852e-06,-1.798526e-08,307.860107,349.829712,...,0.001963,0.008806,-0.032558,-0.053620,-0.060068,-0.034669,-0.014259,-0.003236,0.000338,0.001661
106,1011.150000,14.735302,1.320104e-07,-0.000006,-2.592002e-06,1.388817e-06,4.577279e-06,3.143935e-06,308.141876,349.570618,...,-0.005751,-0.015676,-0.066960,-0.084195,-0.076993,-0.058855,-0.012383,0.002143,0.003600,0.003575
107,1010.911250,14.472641,8.215622e-06,-0.000002,-3.117607e-07,6.945216e-06,-3.222564e-07,-7.365141e-07,310.461548,349.037231,...,-0.011831,-0.029064,-0.079038,-0.099321,-0.084296,-0.056213,-0.022702,-0.011650,-0.005753,-0.003370
108,1010.197500,13.135821,-8.778028e-07,0.000008,-3.190881e-06,2.965728e-08,-3.890142e-06,-2.785965e-06,308.148254,348.835663,...,-0.010985,-0.025936,-0.060773,-0.088269,-0.065035,-0.024801,0.001298,0.004623,0.005371,0.005806


In [33]:
df1.to_csv(output+'timeseries_all_wp_chanhom.csv')

In [34]:
#storm3 saudel
maindf2=pd.read_csv(output+'inner/inner_wp_all_saudel.csv',delimiter=r",")
maindf2.rename({"Unnamed: 0":"a"}, axis="columns", inplace=True)
maindf2=maindf2.drop('a', axis=1)

pdf_saudel=pd.read_csv(target+'2020_pmin_nwp_saudel.csv',delimiter=r",")
pdf_saudel.rename({"Unnamed: 0":"a"}, axis="columns", inplace=True)
pdf_saudel=pdf_saudel.drop('a', axis=1)

extra=pd.read_csv(output+'nwp/outder_2020_nwp_saudel.csv',delimiter=r",")
extra.rename({"Unnamed: 0":"a"}, axis="columns", inplace=True)
extra=extra.drop('a', axis=1)

rlev_saudel=pd.DataFrame.from_dict(TCr_ts[2])
prlev_saudel=pd.DataFrame.from_dict(TCpr_ts[2])
slev_saudel=pd.DataFrame.from_dict(TCs_ts[2])

prlev_saudel['outmslp']=prlev_saudel['outmslp']/100

ts1_saudel=pd.concat([maindf2,extra,slev_saudel,rlev_saudel,prlev_saudel], axis=1, join='inner')


In [35]:
tempvortdict = {'outvort_'+plev[i]:TCp_ts[2]['outvort'][:,i] for i in range(21)}
temprhumdict = {'outrhum_'+plev[i]:TCp_ts[2]['outrhum'][:,i] for i in range(21)}
tempgpotdict = {'outgpot_'+plev[i]:TCgp_ts[2]['outgpot'][:,i] for i in range(21)}
tempvveldict = {'outvvel_'+preslv[i]:TCvv_ts[2]['outvvel'][:,i] for i in range(18)}
dict1 = {**tempvortdict,**temprhumdict}
dict3 = {**tempgpotdict,**tempvveldict}

alldict = {**dict1,**dict3}
plev_saudel=pd.DataFrame.from_dict(alldict)
ts_saudel=pd.concat([pdf_saudel,ts1_saudel,plev_saudel], axis=1, join='inner')
ts_saudel

,Pmin,w10max,div100,div200,div250,div300,div400,div500,eqt1000,eqt200,...,outvvel_250,outvvel_300,outvvel_400,outvvel_500,outvvel_600,outvvel_700,outvvel_850,outvvel_925,outvvel_975,outvvel_1000
0,1006.465000,9.720530,3.659137e-05,2.674120e-05,0.000021,1.140453e-05,-1.782170e-05,-2.513128e-05,336.903381,351.404266,...,-0.113815,-0.114640,-0.114279,-0.108541,-0.101855,-0.067678,-0.024106,-0.009753,0.001967,0.008017
1,1005.538750,10.586460,3.764753e-05,1.872544e-05,0.000009,1.730541e-07,-1.574785e-05,-1.932878e-05,335.100098,351.041016,...,-0.114558,-0.118105,-0.115824,-0.111746,-0.101842,-0.071088,-0.043072,-0.034260,-0.024335,-0.019181
2,1004.002500,11.196884,3.723363e-05,1.083731e-05,-0.000004,-6.414994e-06,-6.280767e-06,-5.658326e-06,337.218567,350.555145,...,-0.109829,-0.118167,-0.120095,-0.114711,-0.100966,-0.072908,-0.038719,-0.024326,-0.012555,-0.006368
3,1005.632500,9.689566,4.320152e-05,2.101832e-05,0.000006,-1.290145e-06,9.780704e-07,-1.113622e-06,337.203796,351.444580,...,-0.091210,-0.103357,-0.100499,-0.084754,-0.064945,-0.040273,-0.010425,0.002160,0.012273,0.017810
4,1006.786875,9.854545,3.589010e-05,2.179953e-05,0.000012,3.392242e-06,5.450032e-06,-6.786525e-06,336.987823,352.273590,...,-0.099417,-0.117290,-0.117297,-0.101189,-0.083435,-0.058615,-0.029144,-0.014219,-0.001282,0.005806
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
74,1006.719375,12.631915,-9.574364e-07,-6.402127e-06,0.000005,-3.307617e-07,-2.609506e-06,3.778701e-06,333.232300,352.130737,...,-0.007944,0.000505,-0.007480,-0.017832,-0.025294,-0.024590,-0.032436,-0.016585,-0.001468,0.005229
75,1007.193125,12.232833,-3.464161e-07,-5.503065e-06,0.000003,-3.078860e-06,-1.626411e-06,5.109677e-06,330.830139,351.811157,...,-0.001606,0.003491,0.000651,-0.007249,-0.012339,-0.015091,-0.015156,0.004973,0.020816,0.027480
76,1009.588750,11.489685,5.000711e-06,-8.724845e-07,-0.000003,-4.128350e-06,1.111870e-06,3.750386e-07,329.997711,351.475037,...,0.021707,0.026788,0.029720,0.022458,0.010667,0.000097,0.002918,0.022782,0.034005,0.038176
77,1010.318750,10.603266,-6.748751e-07,-6.796029e-07,-0.000003,-4.094268e-06,5.779794e-06,-3.246847e-06,326.219360,351.265808,...,-0.011781,-0.013716,-0.015099,-0.027611,-0.044834,-0.057522,-0.039904,-0.019011,-0.014439,-0.013758


In [36]:
df1=ts_saudel
df1=df1.drop('ws10', axis=1)
df1=df1.drop('u10', axis=1)
df1=df1.drop('v10', axis=1)
df1=df1.drop('out_ws10', axis=1)
df1=df1.drop('outu10', axis=1)
df1=df1.drop('outv10', axis=1)
df1=df1.drop('vi_div_ke_flux', axis=1)
df1=df1.drop('vi_div_gpot_flux', axis=1)
df1=df1.drop('vi_ke', axis=1)
df1=df1.drop('vi_div_olr_flux', axis=1)
df1=df1.drop('vi_temp', axis=1)
df1=df1.drop('inst_10m_wnd_gst', axis=1)
df1=df1.drop('surfmean_swr_flux', axis=1)
df1=df1.drop('surfmean_lwr_flux', axis=1)
df1=df1.drop('dwnwrdmean_swr_flux', axis=1)
df1=df1.drop('topmean_lwr_flux', axis=1)
df1=df1.drop('topmean_swr_flux', axis=1)
df1=df1.drop('outvi_div_ke_flux', axis=1)
df1=df1.drop('outvi_div_gpot_flux', axis=1)
df1=df1.drop('outvi_ke', axis=1)
df1=df1.drop('outvi_div_olr_flux', axis=1)
df1=df1.drop('outvi_temp', axis=1)
df1=df1.drop('outinst_10m_wnd_gst', axis=1)
df1=df1.drop('outsurfmean_swr_flux', axis=1)
df1=df1.drop('outsurfmean_lwr_flux', axis=1)
df1=df1.drop('outdwnwrdmean_swr_flux', axis=1)
df1=df1.drop('outtopmean_lwr_flux', axis=1)
df1=df1.drop('outtopmean_swr_flux', axis=1)
df1=df1.drop('outmslp', axis=1)
df1=df1.drop('mslp', axis=1)

In [37]:
df1

,Pmin,w10max,div100,div200,div250,div300,div400,div500,eqt1000,eqt200,...,outvvel_250,outvvel_300,outvvel_400,outvvel_500,outvvel_600,outvvel_700,outvvel_850,outvvel_925,outvvel_975,outvvel_1000
0,1006.465000,9.720530,3.659137e-05,2.674120e-05,0.000021,1.140453e-05,-1.782170e-05,-2.513128e-05,336.903381,351.404266,...,-0.113815,-0.114640,-0.114279,-0.108541,-0.101855,-0.067678,-0.024106,-0.009753,0.001967,0.008017
1,1005.538750,10.586460,3.764753e-05,1.872544e-05,0.000009,1.730541e-07,-1.574785e-05,-1.932878e-05,335.100098,351.041016,...,-0.114558,-0.118105,-0.115824,-0.111746,-0.101842,-0.071088,-0.043072,-0.034260,-0.024335,-0.019181
2,1004.002500,11.196884,3.723363e-05,1.083731e-05,-0.000004,-6.414994e-06,-6.280767e-06,-5.658326e-06,337.218567,350.555145,...,-0.109829,-0.118167,-0.120095,-0.114711,-0.100966,-0.072908,-0.038719,-0.024326,-0.012555,-0.006368
3,1005.632500,9.689566,4.320152e-05,2.101832e-05,0.000006,-1.290145e-06,9.780704e-07,-1.113622e-06,337.203796,351.444580,...,-0.091210,-0.103357,-0.100499,-0.084754,-0.064945,-0.040273,-0.010425,0.002160,0.012273,0.017810
4,1006.786875,9.854545,3.589010e-05,2.179953e-05,0.000012,3.392242e-06,5.450032e-06,-6.786525e-06,336.987823,352.273590,...,-0.099417,-0.117290,-0.117297,-0.101189,-0.083435,-0.058615,-0.029144,-0.014219,-0.001282,0.005806
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
74,1006.719375,12.631915,-9.574364e-07,-6.402127e-06,0.000005,-3.307617e-07,-2.609506e-06,3.778701e-06,333.232300,352.130737,...,-0.007944,0.000505,-0.007480,-0.017832,-0.025294,-0.024590,-0.032436,-0.016585,-0.001468,0.005229
75,1007.193125,12.232833,-3.464161e-07,-5.503065e-06,0.000003,-3.078860e-06,-1.626411e-06,5.109677e-06,330.830139,351.811157,...,-0.001606,0.003491,0.000651,-0.007249,-0.012339,-0.015091,-0.015156,0.004973,0.020816,0.027480
76,1009.588750,11.489685,5.000711e-06,-8.724845e-07,-0.000003,-4.128350e-06,1.111870e-06,3.750386e-07,329.997711,351.475037,...,0.021707,0.026788,0.029720,0.022458,0.010667,0.000097,0.002918,0.022782,0.034005,0.038176
77,1010.318750,10.603266,-6.748751e-07,-6.796029e-07,-0.000003,-4.094268e-06,5.779794e-06,-3.246847e-06,326.219360,351.265808,...,-0.011781,-0.013716,-0.015099,-0.027611,-0.044834,-0.057522,-0.039904,-0.019011,-0.014439,-0.013758


In [38]:
df1.to_csv(output+'timeseries_all_wp_saudel.csv')

In [39]:
#storm 4 molave
maindf2=pd.read_csv(output+'inner/inner_wp_all_molave.csv',delimiter=r",")
maindf2.rename({"Unnamed: 0":"a"}, axis="columns", inplace=True)
maindf2=maindf2.drop('a', axis=1)

pdf_molave=pd.read_csv(target+'2020_pmin_nwp_molave.csv',delimiter=r",")
pdf_molave.rename({"Unnamed: 0":"a"}, axis="columns", inplace=True)
pdf_molave=pdf_molave.drop('a', axis=1)

extra=pd.read_csv(output+'nwp/outder_2020_nwp_molave.csv',delimiter=r",")
extra.rename({"Unnamed: 0":"a"}, axis="columns", inplace=True)
extra=extra.drop('a', axis=1)

rlev_molave=pd.DataFrame.from_dict(TCr_ts[3])
prlev_molave=pd.DataFrame.from_dict(TCpr_ts[3])
slev_molave=pd.DataFrame.from_dict(TCs_ts[3])

prlev_molave['outmslp']=prlev_molave['outmslp']/100

ts1_molave=pd.concat([maindf2,extra,slev_molave,rlev_molave,prlev_molave], axis=1, join='inner')

tempvortdict = {'outvort_'+plev[i]:TCp_ts[3]['outvort'][:,i] for i in range(21)}
temprhumdict = {'outrhum_'+plev[i]:TCp_ts[3]['outrhum'][:,i] for i in range(21)}
tempgpotdict = {'outgpot_'+plev[i]:TCgp_ts[3]['outgpot'][:,i] for i in range(21)}
tempvveldict = {'outvvel_'+preslv[i]:TCvv_ts[3]['outvvel'][:,i] for i in range(18)}
dict1 = {**tempvortdict,**temprhumdict}
dict3 = {**tempgpotdict,**tempvveldict}

alldict = {**dict1,**dict3}
plev_molave=pd.DataFrame.from_dict(alldict)
ts_molave=pd.concat([pdf_molave,ts1_molave,plev_molave], axis=1, join='inner')
ts_molave

,Pmin,w10max,div100,div200,div250,div300,div400,div500,eqt1000,eqt200,...,outvvel_250,outvvel_300,outvvel_400,outvvel_500,outvvel_600,outvvel_700,outvvel_850,outvvel_925,outvvel_975,outvvel_1000
0,1005.517500,12.301943,2.422013e-05,9.087721e-06,1.454434e-05,1.568486e-05,1.927993e-05,6.591382e-06,329.807648,350.105865,...,-0.067471,-0.071853,-0.058298,-0.036908,-0.018958,0.015757,0.040894,0.025238,0.007422,-0.001608
1,1005.473125,12.164107,3.737096e-05,9.660725e-06,1.567254e-05,1.686938e-05,1.164998e-05,-5.515210e-06,335.456177,350.560333,...,-0.065571,-0.069438,-0.053106,-0.030871,-0.018449,0.014615,0.046851,0.039445,0.026296,0.019404
2,1007.500000,11.694481,2.488430e-05,-7.963322e-09,5.783167e-06,2.563052e-06,1.264052e-05,-1.959789e-06,335.505066,349.713928,...,-0.074378,-0.085894,-0.077754,-0.049189,-0.036528,-0.014653,0.011390,0.013241,0.005875,0.001237
3,1006.016250,12.752129,1.998777e-05,5.528326e-06,1.065627e-05,2.069961e-06,7.551805e-06,-6.284647e-06,334.652679,349.882324,...,-0.067649,-0.077406,-0.069147,-0.046383,-0.031372,-0.012700,0.007836,0.002913,-0.008574,-0.014943
4,1004.917500,12.502798,2.382553e-05,1.102856e-05,9.448201e-06,3.950343e-06,4.123305e-06,-8.472813e-06,330.993011,350.502472,...,-0.046206,-0.053866,-0.041525,-0.024396,-0.006886,0.011746,0.031026,0.022378,0.011059,0.005076
5,1005.830000,14.583197,3.810575e-05,1.015933e-05,4.381408e-06,1.098954e-06,1.560478e-06,-1.099295e-05,334.334595,351.017334,...,-0.026456,-0.029141,-0.016669,-0.011318,0.002108,0.019729,0.040769,0.033601,0.020629,0.012733
6,1007.749375,14.204258,2.357032e-05,4.712616e-06,5.480355e-07,2.281235e-06,9.674590e-06,1.300695e-06,329.677490,350.075378,...,-0.014169,-0.022499,-0.013417,-0.010640,0.002301,0.012795,0.016995,0.014080,0.009451,0.005542
7,1006.738125,16.525116,4.617942e-05,4.350195e-05,2.196796e-05,1.114561e-05,6.170811e-06,-6.105243e-06,333.726379,350.572815,...,0.011731,0.011050,0.018600,0.018764,0.020845,0.010173,-0.007346,-0.011537,-0.013341,-0.015194
8,1004.627500,15.001616,6.947882e-05,4.550481e-05,2.771771e-05,1.312502e-05,-1.197278e-06,-1.988836e-05,336.777283,352.596680,...,0.013618,0.011583,0.015786,0.011917,0.010034,-0.006709,-0.013831,-0.008318,-0.003592,-0.001837
9,1005.108750,14.950413,6.871610e-05,3.032470e-05,1.139088e-05,-3.876331e-06,-2.267570e-05,-2.569791e-05,337.740601,352.643738,...,-0.015042,-0.027076,-0.043514,-0.054593,-0.052786,-0.052699,-0.022702,-0.002325,0.009349,0.014454


In [40]:
df1=ts_molave
df1=df1.drop('ws10', axis=1)
df1=df1.drop('u10', axis=1)
df1=df1.drop('v10', axis=1)
df1=df1.drop('out_ws10', axis=1)
df1=df1.drop('outu10', axis=1)
df1=df1.drop('outv10', axis=1)
df1=df1.drop('vi_div_ke_flux', axis=1)
df1=df1.drop('vi_div_gpot_flux', axis=1)
df1=df1.drop('vi_ke', axis=1)
df1=df1.drop('vi_div_olr_flux', axis=1)
df1=df1.drop('vi_temp', axis=1)
df1=df1.drop('inst_10m_wnd_gst', axis=1)
df1=df1.drop('surfmean_swr_flux', axis=1)
df1=df1.drop('surfmean_lwr_flux', axis=1)
df1=df1.drop('dwnwrdmean_swr_flux', axis=1)
df1=df1.drop('topmean_lwr_flux', axis=1)
df1=df1.drop('topmean_swr_flux', axis=1)
df1=df1.drop('outvi_div_ke_flux', axis=1)
df1=df1.drop('outvi_div_gpot_flux', axis=1)
df1=df1.drop('outvi_ke', axis=1)
df1=df1.drop('outvi_div_olr_flux', axis=1)
df1=df1.drop('outvi_temp', axis=1)
df1=df1.drop('outinst_10m_wnd_gst', axis=1)
df1=df1.drop('outsurfmean_swr_flux', axis=1)
df1=df1.drop('outsurfmean_lwr_flux', axis=1)
df1=df1.drop('outdwnwrdmean_swr_flux', axis=1)
df1=df1.drop('outtopmean_lwr_flux', axis=1)
df1=df1.drop('outtopmean_swr_flux', axis=1)
df1=df1.drop('outmslp', axis=1)
df1=df1.drop('mslp', axis=1)

In [41]:
df1.to_csv(output+'timeseries_all_wp_molave.csv')

In [42]:
#storm 5 goni
maindf2=pd.read_csv(output+'inner/inner_wp_all_goni.csv',delimiter=r",")
maindf2.rename({"Unnamed: 0":"a"}, axis="columns", inplace=True)
maindf2=maindf2.drop('a', axis=1)

pdf_goni=pd.read_csv(target+'2020_pmin_nwp_goni.csv',delimiter=r",")
pdf_goni.rename({"Unnamed: 0":"a"}, axis="columns", inplace=True)
pdf_goni=pdf_goni.drop('a', axis=1)

extra=pd.read_csv(output+'nwp/outder_2020_nwp_goni.csv',delimiter=r",")
extra.rename({"Unnamed: 0":"a"}, axis="columns", inplace=True)
extra=extra.drop('a', axis=1) 


rlev_goni=pd.DataFrame.from_dict(TCr_ts[4])
prlev_goni=pd.DataFrame.from_dict(TCpr_ts[4])
slev_goni=pd.DataFrame.from_dict(TCs_ts[4])

prlev_goni['outmslp']=prlev_goni['outmslp']/100

ts1_goni=pd.concat([maindf2,extra,slev_goni,rlev_goni,prlev_goni], axis=1, join='inner')

tempvortdict = {'outvort_'+plev[i]:TCp_ts[4]['outvort'][:,i] for i in range(21)}
temprhumdict = {'outrhum_'+plev[i]:TCp_ts[4]['outrhum'][:,i] for i in range(21)}
tempgpotdict = {'outgpot_'+plev[i]:TCgp_ts[4]['outgpot'][:,i] for i in range(21)}
tempvveldict = {'outvvel_'+preslv[i]:TCvv_ts[4]['outvvel'][:,i] for i in range(18)}
dict1 = {**tempvortdict,**temprhumdict}
dict3 = {**tempgpotdict,**tempvveldict}

alldict = {**dict1,**dict3}
plev_goni=pd.DataFrame.from_dict(alldict)
ts_goni=pd.concat([pdf_goni,ts1_goni,plev_goni], axis=1, join='inner')
ts_goni

,Pmin,w10max,div100,div200,div250,div300,div400,div500,eqt1000,eqt200,...,outvvel_250,outvvel_300,outvvel_400,outvvel_500,outvvel_600,outvvel_700,outvvel_850,outvvel_925,outvvel_975,outvvel_1000
0,1008.113750,8.367717,0.000030,1.590303e-05,7.356431e-06,1.214419e-06,5.962330e-07,-1.424135e-06,331.848175,351.363892,...,-0.064497,-0.077735,-0.096250,-0.086866,-0.084769,-0.067712,-0.021602,-0.006876,0.001950,0.006084
1,1008.920625,9.449281,0.000030,1.827138e-05,2.173332e-06,5.862377e-07,-1.973572e-06,-8.502179e-07,330.939117,350.243774,...,-0.059211,-0.070238,-0.081547,-0.058240,-0.048806,-0.031431,0.005894,0.013011,0.015687,0.016980
2,1010.233750,12.327559,0.000009,2.140960e-05,1.090387e-05,7.192536e-06,-5.287471e-06,-1.468169e-06,333.098633,348.851044,...,-0.042166,-0.052879,-0.071344,-0.054969,-0.047487,-0.033512,-0.006427,-0.004406,-0.003249,-0.002745
3,1008.959375,12.489228,0.000018,2.562352e-05,1.746218e-05,1.149249e-05,-1.743597e-06,3.527325e-07,331.694092,349.465851,...,-0.031418,-0.041030,-0.071109,-0.056107,-0.043094,-0.034281,-0.019542,-0.019075,-0.018923,-0.018707
4,1007.684375,11.228815,0.000037,2.299392e-05,2.092692e-05,6.591619e-06,-1.336298e-07,-7.195069e-07,333.489990,349.515747,...,-0.025402,-0.036699,-0.079313,-0.066737,-0.042778,-0.023054,-0.002369,0.000384,-0.000052,-0.001323
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
92,1007.480000,12.553186,0.000015,-1.735051e-06,-1.315080e-05,-1.676779e-05,-4.615766e-06,8.291820e-06,330.858582,350.205811,...,0.000718,0.002871,0.026354,0.035344,0.023464,-0.002582,-0.022818,-0.024104,-0.020235,-0.015333
93,1007.020000,12.279484,0.000018,2.420765e-06,-8.579384e-06,-1.973723e-05,1.458764e-06,1.517077e-05,330.239532,349.540497,...,0.025823,0.027828,0.037811,0.036710,0.019271,-0.003901,-0.010207,-0.003523,0.005037,0.010916
94,1009.248750,10.930732,0.000007,-4.595220e-07,1.057944e-07,-7.029741e-06,3.092277e-06,3.905538e-06,324.859802,350.248230,...,0.042673,0.043550,0.031314,0.004942,-0.021683,-0.030679,-0.015098,-0.001381,0.011978,0.018843
95,1011.123125,11.166134,0.000002,-6.647833e-06,-3.899162e-06,-6.179701e-06,8.060028e-06,4.362748e-06,320.559174,350.367462,...,0.038647,0.039469,0.019497,-0.018908,-0.044562,-0.051324,-0.037847,-0.022430,-0.013220,-0.009114


In [43]:
df1=ts_goni
df1=df1.drop('ws10', axis=1)
df1=df1.drop('u10', axis=1)
df1=df1.drop('v10', axis=1)
df1=df1.drop('out_ws10', axis=1)
df1=df1.drop('outu10', axis=1)
df1=df1.drop('outv10', axis=1)
df1=df1.drop('vi_div_ke_flux', axis=1)
df1=df1.drop('vi_div_gpot_flux', axis=1)
df1=df1.drop('vi_ke', axis=1)
df1=df1.drop('vi_div_olr_flux', axis=1)
df1=df1.drop('vi_temp', axis=1)
df1=df1.drop('inst_10m_wnd_gst', axis=1)
df1=df1.drop('surfmean_swr_flux', axis=1)
df1=df1.drop('surfmean_lwr_flux', axis=1)
df1=df1.drop('dwnwrdmean_swr_flux', axis=1)
df1=df1.drop('topmean_lwr_flux', axis=1)
df1=df1.drop('topmean_swr_flux', axis=1)
df1=df1.drop('outvi_div_ke_flux', axis=1)
df1=df1.drop('outvi_div_gpot_flux', axis=1)
df1=df1.drop('outvi_ke', axis=1)
df1=df1.drop('outvi_div_olr_flux', axis=1)
df1=df1.drop('outvi_temp', axis=1)
df1=df1.drop('outinst_10m_wnd_gst', axis=1)
df1=df1.drop('outsurfmean_swr_flux', axis=1)
df1=df1.drop('outsurfmean_lwr_flux', axis=1)
df1=df1.drop('outdwnwrdmean_swr_flux', axis=1)
df1=df1.drop('outtopmean_lwr_flux', axis=1)
df1=df1.drop('outtopmean_swr_flux', axis=1)
df1=df1.drop('outmslp', axis=1)
df1=df1.drop('mslp', axis=1)

In [44]:
df1

,Pmin,w10max,div100,div200,div250,div300,div400,div500,eqt1000,eqt200,...,outvvel_250,outvvel_300,outvvel_400,outvvel_500,outvvel_600,outvvel_700,outvvel_850,outvvel_925,outvvel_975,outvvel_1000
0,1008.113750,8.367717,0.000030,1.590303e-05,7.356431e-06,1.214419e-06,5.962330e-07,-1.424135e-06,331.848175,351.363892,...,-0.064497,-0.077735,-0.096250,-0.086866,-0.084769,-0.067712,-0.021602,-0.006876,0.001950,0.006084
1,1008.920625,9.449281,0.000030,1.827138e-05,2.173332e-06,5.862377e-07,-1.973572e-06,-8.502179e-07,330.939117,350.243774,...,-0.059211,-0.070238,-0.081547,-0.058240,-0.048806,-0.031431,0.005894,0.013011,0.015687,0.016980
2,1010.233750,12.327559,0.000009,2.140960e-05,1.090387e-05,7.192536e-06,-5.287471e-06,-1.468169e-06,333.098633,348.851044,...,-0.042166,-0.052879,-0.071344,-0.054969,-0.047487,-0.033512,-0.006427,-0.004406,-0.003249,-0.002745
3,1008.959375,12.489228,0.000018,2.562352e-05,1.746218e-05,1.149249e-05,-1.743597e-06,3.527325e-07,331.694092,349.465851,...,-0.031418,-0.041030,-0.071109,-0.056107,-0.043094,-0.034281,-0.019542,-0.019075,-0.018923,-0.018707
4,1007.684375,11.228815,0.000037,2.299392e-05,2.092692e-05,6.591619e-06,-1.336298e-07,-7.195069e-07,333.489990,349.515747,...,-0.025402,-0.036699,-0.079313,-0.066737,-0.042778,-0.023054,-0.002369,0.000384,-0.000052,-0.001323
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
92,1007.480000,12.553186,0.000015,-1.735051e-06,-1.315080e-05,-1.676779e-05,-4.615766e-06,8.291820e-06,330.858582,350.205811,...,0.000718,0.002871,0.026354,0.035344,0.023464,-0.002582,-0.022818,-0.024104,-0.020235,-0.015333
93,1007.020000,12.279484,0.000018,2.420765e-06,-8.579384e-06,-1.973723e-05,1.458764e-06,1.517077e-05,330.239532,349.540497,...,0.025823,0.027828,0.037811,0.036710,0.019271,-0.003901,-0.010207,-0.003523,0.005037,0.010916
94,1009.248750,10.930732,0.000007,-4.595220e-07,1.057944e-07,-7.029741e-06,3.092277e-06,3.905538e-06,324.859802,350.248230,...,0.042673,0.043550,0.031314,0.004942,-0.021683,-0.030679,-0.015098,-0.001381,0.011978,0.018843
95,1011.123125,11.166134,0.000002,-6.647833e-06,-3.899162e-06,-6.179701e-06,8.060028e-06,4.362748e-06,320.559174,350.367462,...,0.038647,0.039469,0.019497,-0.018908,-0.044562,-0.051324,-0.037847,-0.022430,-0.013220,-0.009114


In [45]:
df1.to_csv(output+'timeseries_all_wp_goni.csv')

In [46]:
#storm 6 atsani
maindf2=pd.read_csv(output+'inner/inner_wp_all_atsani.csv',delimiter=r",")
maindf2.rename({"Unnamed: 0":"a"}, axis="columns", inplace=True)
maindf2=maindf2.drop('a', axis=1)

pdf_atsani=pd.read_csv(target+'2020_pmin_nwp_atsani.csv',delimiter=r",")
pdf_atsani.rename({"Unnamed: 0":"a"}, axis="columns", inplace=True)
pdf_atsani=pdf_atsani.drop('a', axis=1)

extra=pd.read_csv(output+'nwp/outder_2020_nwp_atsani.csv',delimiter=r",")
extra.rename({"Unnamed: 0":"a"}, axis="columns", inplace=True)
extra=extra.drop('a', axis=1)

rlev_atsani=pd.DataFrame.from_dict(TCr_ts[5])
prlev_atsani=pd.DataFrame.from_dict(TCpr_ts[5])
slev_atsani=pd.DataFrame.from_dict(TCs_ts[5])

prlev_atsani['outmslp']=prlev_atsani['outmslp']/100

ts1_atsani=pd.concat([maindf2,extra,slev_atsani,rlev_atsani,prlev_atsani], axis=1, join='inner')

tempvortdict = {'outvort_'+plev[i]:TCp_ts[5]['outvort'][:,i] for i in range(21)}
temprhumdict = {'outrhum_'+plev[i]:TCp_ts[5]['outrhum'][:,i] for i in range(21)}
tempgpotdict = {'outgpot_'+plev[i]:TCgp_ts[5]['outgpot'][:,i] for i in range(21)}
tempvveldict = {'outvvel_'+preslv[i]:TCvv_ts[5]['outvvel'][:,i] for i in range(18)}
dict1 = {**tempvortdict,**temprhumdict}
dict3 = {**tempgpotdict,**tempvveldict}

alldict = {**dict1,**dict3}
plev_atsani=pd.DataFrame.from_dict(alldict)
ts_atsani=pd.concat([pdf_atsani,ts1_atsani,plev_atsani], axis=1, join='inner')
ts_atsani

,Pmin,w10max,div100,div200,div250,div300,div400,div500,eqt1000,eqt200,...,outvvel_250,outvvel_300,outvvel_400,outvvel_500,outvvel_600,outvvel_700,outvvel_850,outvvel_925,outvvel_975,outvvel_1000
0,1006.757500,13.654966,3.250962e-05,8.415326e-06,-1.079934e-05,-1.733789e-06,0.000004,-5.103514e-06,337.551910,350.967773,...,-0.108892,-0.128866,-0.133397,-0.112259,-0.090409,-0.078572,-0.050334,-0.030261,-0.012168,-0.003085
1,1004.320000,13.333771,3.800994e-05,6.226319e-06,-1.114594e-05,3.774389e-06,0.000008,-1.716448e-06,338.075439,351.065887,...,-0.086940,-0.103901,-0.109243,-0.097045,-0.085665,-0.078064,-0.048879,-0.037005,-0.027253,-0.022106
2,1003.313750,13.895884,4.568494e-05,1.353166e-05,-5.222186e-07,7.107878e-06,0.000010,-7.376450e-06,336.909515,351.134125,...,-0.067728,-0.078058,-0.080313,-0.071530,-0.057701,-0.041742,-0.008973,-0.000503,0.002879,0.004572
3,1005.125625,14.529068,5.294114e-05,1.938432e-05,1.272257e-05,1.179825e-05,0.000006,-1.222570e-05,335.190125,351.260529,...,-0.043849,-0.055263,-0.050753,-0.038137,-0.025635,-0.005758,0.023603,0.023894,0.020325,0.019375
4,1006.505625,14.130989,4.391254e-05,2.829460e-05,1.903524e-05,4.756615e-06,-0.000016,-2.586564e-06,334.479858,351.155609,...,-0.044852,-0.054444,-0.053192,-0.024984,-0.001592,0.003411,0.004917,-0.002270,-0.006203,-0.008020
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
69,1012.336250,14.715964,4.517028e-07,-2.781457e-06,-1.505086e-06,3.793326e-06,0.000010,-2.253578e-06,321.156982,352.486633,...,0.002535,-0.002014,-0.015556,-0.018767,-0.023355,-0.030369,-0.033685,-0.028518,-0.025134,-0.024883
70,1009.863125,15.020427,-2.634104e-06,3.636680e-06,-8.155421e-07,3.587869e-06,0.000008,3.420132e-08,318.570557,352.428070,...,-0.015022,-0.023897,-0.031515,-0.023081,-0.023656,-0.024831,-0.011571,-0.007330,-0.012136,-0.016336
71,1009.736250,14.803987,-3.647683e-06,8.870117e-06,-5.950974e-07,-2.493488e-06,0.000004,2.765601e-06,315.623291,352.288818,...,-0.009956,-0.019137,-0.022989,-0.007434,-0.008151,-0.003633,0.017717,0.015105,0.010533,0.008072
72,1010.506875,13.906894,-2.137216e-06,5.670727e-07,3.796495e-06,-6.555103e-08,-0.000002,-3.116218e-08,314.585815,352.522705,...,-0.019821,-0.025540,-0.017526,0.005282,0.006797,0.010731,0.006804,0.002153,0.001252,0.003483


In [47]:
df1=ts_atsani
df1=df1.drop('ws10', axis=1)
df1=df1.drop('u10', axis=1)
df1=df1.drop('v10', axis=1)
df1=df1.drop('out_ws10', axis=1)
df1=df1.drop('outu10', axis=1)
df1=df1.drop('outv10', axis=1)
df1=df1.drop('vi_div_ke_flux', axis=1)
df1=df1.drop('vi_div_gpot_flux', axis=1)
df1=df1.drop('vi_ke', axis=1)
df1=df1.drop('vi_div_olr_flux', axis=1)
df1=df1.drop('vi_temp', axis=1)
df1=df1.drop('inst_10m_wnd_gst', axis=1)
df1=df1.drop('surfmean_swr_flux', axis=1)
df1=df1.drop('surfmean_lwr_flux', axis=1)
df1=df1.drop('dwnwrdmean_swr_flux', axis=1)
df1=df1.drop('topmean_lwr_flux', axis=1)
df1=df1.drop('topmean_swr_flux', axis=1)
df1=df1.drop('outvi_div_ke_flux', axis=1)
df1=df1.drop('outvi_div_gpot_flux', axis=1)
df1=df1.drop('outvi_ke', axis=1)
df1=df1.drop('outvi_div_olr_flux', axis=1)
df1=df1.drop('outvi_temp', axis=1)
df1=df1.drop('outinst_10m_wnd_gst', axis=1)
df1=df1.drop('outsurfmean_swr_flux', axis=1)
df1=df1.drop('outsurfmean_lwr_flux', axis=1)
df1=df1.drop('outdwnwrdmean_swr_flux', axis=1)
df1=df1.drop('outtopmean_lwr_flux', axis=1)
df1=df1.drop('outtopmean_swr_flux', axis=1)
df1=df1.drop('outmslp', axis=1)
df1=df1.drop('mslp', axis=1)

In [48]:
df1.to_csv(output+'timeseries_all_wp_atsani.csv')

In [49]:
#storm 7 vamco
maindf2=pd.read_csv(output+'inner/inner_wp_all_vamco.csv',delimiter=r",")
maindf2.rename({"Unnamed: 0":"a"}, axis="columns", inplace=True)
maindf2=maindf2.drop('a', axis=1)

pdf_vamco=pd.read_csv(target+'2020_pmin_nwp_vamco.csv',delimiter=r",")
pdf_vamco.rename({"Unnamed: 0":"a"}, axis="columns", inplace=True)
pdf_vamco=pdf_vamco.drop('a', axis=1)

extra=pd.read_csv(output+'nwp/outder_2020_nwp_vamco.csv',delimiter=r",")
extra.rename({"Unnamed: 0":"a"}, axis="columns", inplace=True)
extra=extra.drop('a', axis=1)

rlev_vamco=pd.DataFrame.from_dict(TCr_ts[6])
prlev_vamco=pd.DataFrame.from_dict(TCpr_ts[6])
slev_vamco=pd.DataFrame.from_dict(TCs_ts[6])

prlev_vamco['outmslp']=prlev_vamco['outmslp']/100

ts1_vamco=pd.concat([maindf2,extra,slev_vamco,rlev_vamco,prlev_vamco], axis=1, join='inner')

tempvortdict = {'outvort_'+plev[i]:TCp_ts[6]['outvort'][:,i] for i in range(21)}
temprhumdict = {'outrhum_'+plev[i]:TCp_ts[6]['outrhum'][:,i] for i in range(21)}
tempgpotdict = {'outgpot_'+plev[i]:TCgp_ts[6]['outgpot'][:,i] for i in range(21)}
tempvveldict = {'outvvel_'+preslv[i]:TCvv_ts[6]['outvvel'][:,i] for i in range(18)}
dict1 = {**tempvortdict,**temprhumdict}
dict3 = {**tempgpotdict,**tempvveldict}

alldict = {**dict1,**dict3}
plev_vamco=pd.DataFrame.from_dict(alldict)
ts_vamco=pd.concat([pdf_vamco,ts1_vamco,plev_vamco], axis=1, join='inner')
ts_vamco

,Pmin,w10max,div100,div200,div250,div300,div400,div500,eqt1000,eqt200,...,outvvel_250,outvvel_300,outvvel_400,outvvel_500,outvvel_600,outvvel_700,outvvel_850,outvvel_925,outvvel_975,outvvel_1000
0,1008.139375,12.301943,0.000034,3.935914e-05,2.301252e-05,-0.000003,-0.000007,-0.000008,335.176880,351.065857,...,-0.108072,-0.136401,-0.162858,-0.158399,-0.119224,-0.072216,-0.028436,-0.013155,-0.000843,0.005923
1,1007.281250,12.164107,0.000030,3.894729e-05,3.033575e-05,0.000009,-0.000004,0.000002,335.679382,351.425781,...,-0.101543,-0.127403,-0.153648,-0.137682,-0.099519,-0.064408,-0.042974,-0.035638,-0.030218,-0.027503
2,1005.983125,11.694481,0.000037,4.089281e-05,3.292633e-05,0.000012,-0.000004,-0.000004,335.140381,352.164673,...,-0.097735,-0.118789,-0.148284,-0.132988,-0.090388,-0.051766,-0.024944,-0.015646,-0.010232,-0.007653
3,1006.540625,12.752129,0.000044,2.951113e-05,1.533757e-05,0.000003,0.000005,-0.000002,335.450409,351.865417,...,-0.106899,-0.127348,-0.152186,-0.131925,-0.087418,-0.050186,-0.020528,-0.006549,0.002084,0.006561
4,1008.196875,12.502798,0.000042,3.501577e-05,3.285955e-05,0.000022,0.000015,-0.000008,335.887512,351.208984,...,-0.110143,-0.122042,-0.126769,-0.116716,-0.078044,-0.047339,-0.016240,-0.005097,-0.000578,0.002069
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62,1011.961250,13.873990,-0.000009,-4.118349e-07,6.354632e-06,0.000002,0.000003,0.000004,307.785950,352.645599,...,-0.001947,-0.014250,-0.003842,-0.001693,0.008652,0.007453,-0.015153,-0.015924,0.011534,0.025228
63,1011.471250,14.389834,-0.000012,1.114982e-06,5.508081e-06,0.000002,0.000004,0.000006,303.966522,352.751221,...,0.004571,-0.009670,-0.003470,-0.000130,0.012333,0.013138,-0.006862,-0.002462,0.025341,0.039663
64,1012.944375,13.112446,-0.000010,-1.615069e-06,-3.792945e-07,-0.000002,0.000005,0.000010,300.972595,353.047821,...,0.014571,0.000968,-0.003358,0.003358,0.013361,0.014282,0.012042,0.020981,0.042444,0.053169
65,1014.429375,12.451212,-0.000010,-3.569903e-06,-4.802502e-07,-0.000006,0.000005,0.000008,298.801605,353.123169,...,0.007081,-0.001068,-0.011161,-0.006752,0.000438,0.001844,-0.009627,-0.015025,-0.009482,-0.004306


In [50]:
df1=ts_vamco
df1=df1.drop('ws10', axis=1)
df1=df1.drop('u10', axis=1)
df1=df1.drop('v10', axis=1)
df1=df1.drop('out_ws10', axis=1)
df1=df1.drop('outu10', axis=1)
df1=df1.drop('outv10', axis=1)
df1=df1.drop('vi_div_ke_flux', axis=1)
df1=df1.drop('vi_div_gpot_flux', axis=1)
df1=df1.drop('vi_ke', axis=1)
df1=df1.drop('vi_div_olr_flux', axis=1)
df1=df1.drop('vi_temp', axis=1)
df1=df1.drop('inst_10m_wnd_gst', axis=1)
df1=df1.drop('surfmean_swr_flux', axis=1)
df1=df1.drop('surfmean_lwr_flux', axis=1)
df1=df1.drop('dwnwrdmean_swr_flux', axis=1)
df1=df1.drop('topmean_lwr_flux', axis=1)
df1=df1.drop('topmean_swr_flux', axis=1)
df1=df1.drop('outvi_div_ke_flux', axis=1)
df1=df1.drop('outvi_div_gpot_flux', axis=1)
df1=df1.drop('outvi_ke', axis=1)
df1=df1.drop('outvi_div_olr_flux', axis=1)
df1=df1.drop('outvi_temp', axis=1)
df1=df1.drop('outinst_10m_wnd_gst', axis=1)
df1=df1.drop('outsurfmean_swr_flux', axis=1)
df1=df1.drop('outsurfmean_lwr_flux', axis=1)
df1=df1.drop('outdwnwrdmean_swr_flux', axis=1)
df1=df1.drop('outtopmean_lwr_flux', axis=1)
df1=df1.drop('outtopmean_swr_flux', axis=1)
df1=df1.drop('outmslp', axis=1)
df1=df1.drop('mslp', axis=1)

In [51]:
df1.to_csv(output+'timeseries_all_wp_vamco.csv')